<div style="border-left: 6px solid #FF8C42; padding:20px; border-radius:10px; font-family:Arial, sans-serif; text-align:center; font-size:28px; font-weight:bold;">
  ⚙️ 04 – Hyperparameter Tuning
</div>


<div style="border-left: 6px solid #27ae60; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Import Libraries and Define Paths</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p style="font-size:18px;">This is the initial block of the rare species image classification project.</p>

  <p>In this section, we perform the following tasks:</p>

  <ul style="line-height: 1.6;">
    <li>📁 <strong>Import libraries</strong> for data manipulation (<code>pandas</code>), file paths (<code>pathlib</code>), and image processing (<code>PIL</code>).</li>
    <li>🖼️ <strong>Apply visual styling</strong> using <code>matplotlib</code> and <code>seaborn</code> to ensure clean and consistent plots.</li>
    <li>📂 <strong>Define the main project directories</strong>, including image folders and the metadata CSV file.</li>
    <li>✅ <strong>Automatic path validation</strong> to ensure all required files and directories exist.</li>
  </ul>

  <p>This setup provides a reliable foundation for safely loading and exploring the dataset.</p>
</div>


In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import keras_tuner as kt
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path
from tabulate import tabulate
from PIL import Image
from IPython.display import display

In [2]:
PROJECT_ROOT = Path().resolve().parent

PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_DIR = PROJECT_ROOT / 'reports'
OUTPUTS_DIR = PROJECT_ROOT / 'output'
LOGS_DIR = OUTPUTS_DIR / 'logs'
PREDICTIONS_DIR = OUTPUTS_DIR / 'predictions'
TRAIN_DIR = PROCESSED_DIR / 'train'
VAL_DIR = PROCESSED_DIR / 'val'
TEST_DIR = PROCESSED_DIR / 'test'

<div style="border-left: 6px solid #27ae60; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Define Parameters</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p>In this section, we define the core parameters that will guide the training process of the model. These include the input image size, batch size, number of training epochs, and the directory structure of the dataset.</p>
  
  <p>Setting these values early ensures consistency across all steps and allows for easier adjustments when experimenting with different model architectures or datasets.</p>
</div>


In [3]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 40

<div style="border-left: 6px solid #27ae60; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📂 Load and Prepare the Dataset</h2>
</div>

<div style="margin-left:60px; padding:10px; font-family:Arial, sans-serif; font-size:16px;"> 
  <p>This section is responsible for loading the processed dataset and preparing it for training and evaluation.</p>

  <p>Using <code>ImageDataGenerator</code>, the images are normalized (pixel values scaled between 0 and 1), and loaded in batches directly from the respective folders for:</p>

  <ul style="line-height: 1.6;">
    <li><strong>🟢 Training set</strong> — used to update model weights during learning</li>
    <li><strong>🟠 Validation set</strong> — used to monitor generalization and prevent overfitting</li>
    <li><strong>🔵 Test set</strong> — used for final evaluation after training</li>
  </ul>

  <p>The dataset is expected to be organized in subfolders where each folder represents one class label.</p>
</div>


In [4]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = datagen.flow_from_directory(TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = datagen.flow_from_directory(VAL_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = datagen.flow_from_directory(TEST_DIR, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

NUM_CLASSES = train_generator.num_classes

Found 8462 images belonging to 202 classes.
Found 2157 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.


<div style="border-left: 6px solid #27ae60; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">⚙️ Set Up Hyperparameter Ranges for MobileNetV2 and VGG16</h2>
</div>

In [5]:
def build_mobilenetv2(hp):
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')
    base_model.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(BatchNormalization())

    model.add(Dense(hp.Int('units1', min_value=128, max_value=1024, step=64), activation='relu'))
    model.add(Dropout(hp.Float('dropout1', 0.3, 0.6, step=0.1)))

    model.add(Dense(hp.Int('units2', min_value=64, max_value=512, step=64), activation='relu'))
    model.add(Dropout(hp.Float('dropout2', 0.2, 0.5, step=0.1)))

    model.add(Dense(NUM_CLASSES, activation='softmax'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 5e-4, 1e-4])
    optimizer = Adam(learning_rate=lr)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
tuner = kt.Hyperband(
    build_mobilenetv2,
    objective='val_accuracy',
    max_epochs=EPOCHS,
    factor=3,
    directory= LOGS_DIR / 'tuner_logs',
    project_name='mobilenetv2_tuning'
)

stop_early = EarlyStopping(monitor='val_loss', patience=3)
tuner.search(train_generator, validation_data=val_generator, epochs=EPOCHS, callbacks=[stop_early])

Reloading Tuner from D:\Repositories\DL_EOLP\output\logs\tuner_logs\mobilenetv2_tuning\tuner0.json


In [7]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

print("Melhores hiperparâmetros encontrados:")
print(best_hyperparams.values)

c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Melhores hiperparâmetros encontrados:
{'units1': 1024, 'dropout1': 0.4, 'units2': 256, 'dropout2': 0.2, 'learning_rate': 0.0005, 'tuner/epochs': 17, 'tuner/initial_epoch': 6, 'tuner/bracket': 3, 'tuner/round': 2, 'tuner/trial_id': '0034'}


In [10]:
def run_mobilenetv2_tuned_pipeline(train_gen, val_gen, test_gen, model_name="mobilenetv2_tuned", image_size=(224, 224), epochs=20):
    models_dir = MODELS_DIR
    logs_dir = LOGS_DIR
    predictions_dir = PREDICTIONS_DIR
    reports_dir = REPORTS_DIR
    figures_dir = reports_dir / "figures"
    
    for d in [models_dir, logs_dir, predictions_dir, figures_dir, reports_dir]:
        d.mkdir(parents=True, exist_ok=True)

    train_generator = train_gen
    val_generator = val_gen
    test_generator = test_gen
    num_classes = train_generator.num_classes

    base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(image_size[0], image_size[1], 3), pooling='avg')
    base_model.trainable = False

    model = Sequential([
        base_model,
        BatchNormalization(),
        Dense(1024, activation='relu'),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
    csv_logger = CSVLogger(logs_dir / f"{model_name}_training_log.csv", append=False)

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[csv_logger, early_stop, reduce_lr]
    )

    model_path = models_dir / f"{model_name}.h5"
    model_weights_path = models_dir / f"{model_name}.weights.h5"
    model.save(model_path)
    model.save_weights(model_weights_path)

    val_loss, val_acc = model.evaluate(val_generator)

    acc_fig_path = figures_dir / f"{model_name}_accuracy_plot.png"
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(acc_fig_path)
    plt.close()

    predictions = model.predict(test_generator)
    predicted_classes = predictions.argmax(axis=1)
    true_classes = test_generator.classes
    class_indices = test_generator.class_indices
    inv_class_indices = {v: k for k, v in class_indices.items()}
    predicted_labels = [inv_class_indices[i] for i in predicted_classes]
    true_labels = [inv_class_indices[i] for i in true_classes]

    report = classification_report(true_classes, predicted_classes, target_names=list(class_indices.keys()), output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_path = reports_dir / f"{model_name}_classification_report.csv"
    report_df.to_csv(report_path)

    heatmap_path = figures_dir / f"{model_name}_classification_report_heatmap_top20.png"
    filtered_df = report_df.drop(["accuracy", "macro avg", "weighted avg"], errors="ignore")
    top_20 = filtered_df.sort_values("support", ascending=False).head(20)
    plt.figure(figsize=(10, 8))
    sns.heatmap(top_20[["precision", "recall", "f1-score"]], annot=True, fmt=".2f", cmap="YlGnBu", linewidths=0.5, annot_kws={"size": 9})
    plt.title("Top 20 Classes – Classification Report", fontsize=14)
    plt.xlabel("Metrics", fontsize=12)
    plt.ylabel("Class", fontsize=12)
    plt.tight_layout()
    plt.savefig(heatmap_path)
    plt.close()

    top_labels = list(top_20.index)
    label_to_index = {name: i for i, name in enumerate(class_indices.keys())}
    top_indices = [label_to_index[l] for l in top_labels]
    filtered_true = [i for i in true_classes if i in top_indices]
    filtered_pred = [p for i, p in enumerate(predicted_classes) if true_classes[i] in top_indices]
    
    cm = confusion_matrix(filtered_true, filtered_pred, labels=top_indices)
    cm_labels = [list(class_indices.keys())[i] for i in top_indices]
    fig, ax = plt.subplots(figsize=(12, 10))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cm_labels)
    disp.plot(ax=ax, xticks_rotation=45, cmap='Blues', colorbar=True)
    plt.title("Confusion Matrix – Top 20 Classes", fontsize=14)
    plt.tight_layout()
    cm_path = figures_dir / f"{model_name}_confusion_matrix_top20.png"
    plt.savefig(cm_path)
    plt.close()

    cm = confusion_matrix(true_classes, predicted_classes)
    fig, ax = plt.subplots(figsize=(20, 20))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(class_indices.keys()))
    disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
    full_cm_path = figures_dir / f"{model_name}_confusion_matrix.png"
    plt.savefig(full_cm_path)
    plt.close()

    filenames = test_generator.filenames
    results_df = pd.DataFrame({
        "filename": filenames,
        "true_label": true_labels,
        "predicted_label": predicted_labels
    })
    pred_path = predictions_dir / f"{model_name}_predictions.csv"
    results_df.to_csv(pred_path, index=False)

    return {
        "model_path": model_path,
        "log_path": logs_dir / f"{model_name}_training_log.csv",
        "report_path": report_path,
        "heatmap_path": heatmap_path,
        "confusion_matrix": full_cm_path,
        "predictions_path": pred_path,
        "accuracy_plot": acc_fig_path,
        "val_accuracy": val_acc
    }

In [11]:
results_mobilenetv2_tuned = run_mobilenetv2_tuned_pipeline(
    train_gen=train_generator,
    val_gen=val_generator,
    test_gen=test_generator,
    model_name="mobilenetv2_tuned",
    image_size=IMAGE_SIZE,
    epochs=EPOCHS
)

print("📦 MobileNetV2 Tuned – Results Summary:\n")
print(f"📁 Model saved at:              {results_mobilenetv2_tuned['model_path']}")
print(f"📄 Training log:                {results_mobilenetv2_tuned['log_path']}")
print(f"📊 Classification report (CSV): {results_mobilenetv2_tuned['report_path']}")
print(f"🧯 Report heatmap (Top 20):     {results_mobilenetv2_tuned['heatmap_path']}")
print(f"📉 Confusion matrix (full):     {results_mobilenetv2_tuned['confusion_matrix']}")
print(f"📈 Accuracy plot:               {results_mobilenetv2_tuned['accuracy_plot']}")
print(f"📑 Predictions CSV:             {results_mobilenetv2_tuned['predictions_path']}")
print(f"✅ Final validation accuracy:   {results_mobilenetv2_tuned['val_accuracy']:.2%}")


Epoch 1/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 43s 147ms/step - accuracy: 0.1324 - loss: 4.6196 - val_accuracy: 0.3922 - val_loss: 2.6700 - learning_rate: 5.0000e-04
Epoch 2/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 38s 142ms/step - accuracy: 0.4230 - loss: 2.3898 - val_accuracy: 0.4798 - val_loss: 2.1675 - learning_rate: 5.0000e-04
Epoch 3/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 38s 143ms/step - accuracy: 0.5785 - loss: 1.6510 - val_accuracy: 0.5192 - val_loss: 1.9781 - learning_rate: 5.0000e-04
Epoch 4/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 43s 164ms/step - accuracy: 0.6657 - loss: 1.2260 - val_accuracy: 0.5364 - val_loss: 1.9839 - learning_rate: 5.0000e-04
Epoch 5/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 43s 163ms/step - accuracy: 0.7395 - loss: 0.9285 - val_accuracy: 0.5633 - val_loss: 1.9464 - learning_rate: 5.0000e-04
Epoch 6/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 45s 168ms/step - accuracy: 0.7809 - loss: 0.7435 - val_accuracy: 0.5517 - val_loss: 2.0373 - learning_rate: 5.0000e-04
Epoch 7/40
265/265 ━━━━━━━━━━━━━━━━━━━━ 43s 16

68/68 ━━━━━━━━━━━━━━━━━━━━ 9s 132ms/step - accuracy: 0.5572 - loss: 2.0058
38/38 ━━━━━━━━━━━━━━━━━━━━ 28s 690ms/step


c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

📦 MobileNetV2 Tuned – Results Summary:

📁 Model saved at:              D:\Repositories\DL_EOLP\models\mobilenetv2_tuned.h5
📄 Training log:                D:\Repositories\DL_EOLP\output\logs\mobilenetv2_tuned_training_log.csv
📊 Classification report (CSV): D:\Repositories\DL_EOLP\reports\mobilenetv2_tuned_classification_report.csv
🧯 Report heatmap (Top 20):     D:\Repositories\DL_EOLP\reports\figures\mobilenetv2_tuned_classification_report_heatmap_top20.png
📉 Confusion matrix (full):     D:\Repositories\DL_EOLP\reports\figures\mobilenetv2_tuned_confusion_matrix.png
📈 Accuracy plot:               D:\Repositories\DL_EOLP\reports\figures\mobilenetv2_tuned_accuracy_plot.png
📑 Predictions CSV:             D:\Repositories\DL_EOLP\output\predictions\mobilenetv2_tuned_predictions.csv
✅ Final validation accuracy:   56.33%
